# Deploy a question-answering model to the Triton Inference Server on NVIDIA Tesla V100s in Azure Kubernetes Service

This notebook shows you how to deploy a Bi-Directional Attention Flow question-ansewring model to the high-performance [Triton Inference Server](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/index.html) on Azure Kubernetes Service (AKS) graphical processing units (GPUs).

Please note that this Public Preview release is subject to the [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/support/legal/preview-supplemental-terms/).

In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

Workspace.create(name='default', subscription_id='6560575d-fa06-4e7d-95fb-f962e74efd7a', resource_group='azureml-examples')

## Preview steps

Necessary only while this feature is in preview, will be unnecessary in a future release of the Azure Machine Learning Python SDK.

In [2]:
from azureml.core.model import Model

Model.Framework.MULTI = "Multi"
Model._SUPPORTED_FRAMEWORKS_FOR_NO_CODE_DEPLOY.append(Model.Framework.MULTI)

## Download model

It's important that your model have this directory structure for Triton Inference Server to be able to load it. [Read more about the directory structure that Triton expects](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html).

In [3]:
import git
import os
import tempfile
import urllib
from azure.storage.blob import BlobClient
from pathlib import Path


url = "https://aka.ms/bidaf-9-model"
response = urllib.request.urlopen(url)

blob_client = BlobClient.from_blob_url(response.url)

# get the root of the repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

# model path
folder_path = prefix.joinpath("models", "triton", "bidaf-9", "1")
model_file_path = prefix.joinpath(folder_path, "model.onnx")

# save the model if it does not already exist
if not os.path.exists(model_file_path):
    os.makedirs(folder_path, exist_ok=True)
    with open(model_file_path, "wb") as my_blob:
        download_stream = blob_client.download_blob()
        my_blob.write(download_stream.readall())

## Register model

A registered model is a logical container stored in the cloud, containing all files located at `model_path`, which is associated with a version number and other metadata.

In [4]:
from azureml.core.model import Model

model_path = prefix.joinpath("models", "triton")

model = Model.register(
    model_path=model_path,
    model_name="bidaf-9-example",
    tags={"area": "Natural language processing", "type": "Question-answering"},
    description="Question answering from ONNX model zoo",
    workspace=ws,
    model_framework=Model.Framework.MULTI
)

print(model.name, model.description, model.version)

Registering model bidaf-9-example
bidaf-9-example Question answering from ONNX model zoo 8


## Deploy webservice

In this case we deploy to a pre-created [AksCompute](https://docs.microsoft.com/python/api/azureml-core/azureml.core.compute.aks.akscompute?view=azure-ml-py#provisioning-configuration-agent-count-none--vm-size-none--ssl-cname-none--ssl-cert-pem-file-none--ssl-key-pem-file-none--location-none--vnet-resourcegroup-name-none--vnet-name-none--subnet-name-none--service-cidr-none--dns-service-ip-none--docker-bridge-cidr-none--cluster-purpose-none--load-balancer-type-none-) called "aks-gpu-deploy," but for other options, see [our documentation](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where?tabs=azcli)


In [5]:
from azureml.core.webservice import AksWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig

service_name = "triton-bidaf-9"

config = AksWebservice.deploy_configuration(
    compute_target_name="aks-gpu-deploy",
    gpu_cores=1,
    cpu_cores=1,
    memory_gb=4, 
    auth_enabled=False
)

service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    deployment_config=config,
    overwrite=True
)

service.wait_for_deployment(show_output=True)

Running.....
Succeeded
AKS service creation operation finished, operation "Succeeded"


## Test the webservice

In [6]:
import json
import sys

# Using a modified version of tritonhttpclient for Preview, PR is out for review
# https://github.com/triton-inference-server/server/pull/2047
sys.path.insert(1, prefix.joinpath("code", "deployment", "triton").__str__())
from bidaf_utils import init, run

init(service.scoring_uri)

data = ["A quick brown fox jumped over the lazy dog.", "Which animal was lower?"]
run(json.dumps(data))
              
              

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Found model: bidaf-9, version: 1,               input meta: [{'name': 'query_char', 'datatype': 'BYTES', 'shape': [-1, 1, 1, 16]}, {'name': 'query_word', 'datatype': 'BYTES', 'shape': [-1, 1]}, {'name': 'context_word', 'datatype': 'BYTES', 'shape': [-1, 1]}, {'name': 'context_char', 'datatype': 'BYTES', 'shape': [-1, 1, 1, 16]}], input config: [{'name': 'query_char', 'data_type': 'TYPE_STRING', 'dims': ['-1', '1', '1', '16']}, {'name': 'query_word', 'data_type': 'TYPE_STRING', 'dims': ['-1', '1']}, {'name': 'context_word', 'data_type': 'TYPE_STRING', 'dims': ['-1', '1']}, {'name': 'context_char', 'data_type': 'TYPE_STRING', 'dims': ['-1', '1', '1', '16']}],               output_meta: [{'name': 'end_pos', 'datatype': 'INT32', 'shape': [1]}, {'name': 'start_pos', 'datatype': 'INT32', 'shape': [1]}], output config: [{'name': 'end_pos', 'data_type': 'TYPE_INT32', 'dims': ['1']}, {'name': 'start_pos', 'data_type': 'TYPE_INT32', 'dims': ['1']}]
Found model: densenet_onnx, version: 1,        

[b'lazy', b'dog']

## Delete the webservice and the downloaded model

In [7]:
service.delete()
os.remove(model_file_path)

# Next steps

Try reading [our documentation](https://aka.ms/triton-aml-docs) to use Triton with your own models or check out the other notebooks in this folder for ways to do pre- and post-processing on the server. 